### Oversampling & prediction
* SMOTE
* Random Oversampling
* rf = RandomForestClassifier(class_weight="balanced") # also add your other parameters!


In [1]:
#imports
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE


import re

import matplotlib.pyplot as plt
import seaborn as sns

# import plotly
# import plotly.tools as tls   
# import plotly.graph_objs as go

In [2]:
import sys
sys.path.append("/Users/Aniket/MyStuff/Study/GreyAtom/Hackathon #3/Hack3_gen_functions")


In [3]:
import Hackathon3GeneralFunctions as genf

In [4]:
data = pd.read_csv('data/train.csv')

In [5]:
data.shape

(7274, 3)

#### Check & drop null tweet

In [6]:
data.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [7]:
data[data.tweet.isnull() == True]

tweet_id tweet  sentiment
1274         6   NaN          1

In [8]:
# data[data.index == 1274]

In [9]:
data.drop(1274,axis=0,inplace=True)

In [10]:
data.isnull().sum()

tweet_id     0
tweet        0
sentiment    0
dtype: int64

#### b

In [11]:
data_og = data.copy()

In [12]:
#using clean text function
data.tweet = data.tweet.map(genf.clean_text)

In [13]:
# data.tweet.head(30)

In [14]:
data['tweet_ts'] = data.tweet.map(genf.tokenize_stem_join)

In [15]:
data.tweet_ts.head(10)

0    sxswnui sxsw appl defin languag of touch with ...
1    learn ab googl doodl all doodl should be light...
2    one of the most in your face ex of steal the s...
3    thi iphon sxsw app would b pretti awesom if it...
4    line outsid the appl store in austin wait for ...
5    technew one lone dude await ipad 2 at appl s s...
6    sxsw tip princ npr video toy shop with zuckerb...
7    nu user rt new ubersoci for iphon now in the a...
8                    free sxsw sampler on itun freemus
9    i think i might go all weekend without see the...
Name: tweet_ts, dtype: object

In [16]:
# len(cv.get_feature_names())

In [17]:
# X = data.tweet_ts
# X.shape

In [18]:
# y = data.sentiment
# y.shape

In [19]:
# X_cv = genf.vectorizer_fit_transform(cv,X)

In [20]:
# X_vec = pd.DataFrame(X_cv)

In [21]:
# len(cv.get_feature_names())

In [22]:
# X_cv.shape

In [23]:
# y.value_counts(normalize=True)*100

In [24]:
# y.value_counts()

#### SMOTE oversampling vanilla

In [25]:
# sm1 = SMOTE(random_state=42)
# X_cv_res, y_res = sm1.fit_resample(X_cv, y)

In [26]:
# y_res.value_counts(normalize=True)*100

#### SMOTE oversampling w {sampling_strategy='minority'}

In [27]:
# sm2 = SMOTE(random_state=42,sampling_strategy='minority')
# X_cv_res2, y_res2 = sm2.fit_resample(X_cv, y)

In [28]:
# y_res2.value_counts(normalize=True)*100

#### SMOTE oversampling w {sampling_strategy='minority'}

In [29]:
# rows = X.shape[0]
# r1 = rows*.6
# r1 = int(r1)
# r1 #1

In [30]:
# sm3 = SMOTE(random_state=42,sampling_strategy={0:r1,1:r1,2:r1,3:r1})
# X_cv_res3, y_res3 = sm3.fit_resample(X_cv, y)

In [31]:
# y_res2.value_counts(normalize=True)*100

In [32]:
# y.value_counts()

In [33]:
# y_res2.value_counts()

In [34]:
# y_res2.shape

In [35]:
data.sentiment.value_counts()

1    4310
2    2382
0     456
3     125
Name: sentiment, dtype: int64

In [36]:
data_wo3 = data[data.sentiment != 3].copy()
# data_wo3.shape

In [37]:
data_wo3.sentiment.value_counts()


1    4310
2    2382
0     456
Name: sentiment, dtype: int64

In [38]:
data_wo3.sentiment.value_counts(normalize=True)*100

1    60.296586
2    33.324007
0     6.379407
Name: sentiment, dtype: float64

In [39]:
total_rows = data_wo3.shape[0]
print("Rows w/o sent==3: ",total_rows)
#for class 1 = 40%
class0_rows = data_wo3.sentiment.value_counts()[1]
print("Rows (class 0) w/o sent==3: ",class0_rows)
total_rows_c0_40 = int(round(class0_rows/0.4))
print("New total Rows total (w/ class 0 = 40%) w/o sent==3: ",total_rows_c0_40)


Rows w/o sent==3:  7148
Rows (class 0) w/o sent==3:  4310
New total Rows total (w/ class 0 = 40%) w/o sent==3:  10775


### ratio 1: 30 40 30


In [40]:
# 4310/.4 #new total Tn1

In [41]:
# Tn1 = 10775 #new total for new ratio
Tn1 = total_rows_c0_40 #new total for new ratio from calculation

ratio_1 = {0:int(round(Tn1*.3)),1:int(round(Tn1*.4)),2:int(round(Tn1*.3))} #new ratio dict
print(ratio_1)
sum(ratio_1.values())

{0: 3232, 1: 4310, 2: 3232}


10774

### ratio 2: 20 40 40 


In [42]:
ratio_2 = {0:int(round(Tn1*.2)),1:int(round(Tn1*.4)),2:int(round(Tn1*.4))} #new ratio dict
ratio_2
print(ratio_2)
sum(ratio_2.values())

{0: 2155, 1: 4310, 2: 4310}


10775

### ratio 3: 25 40 35


In [43]:
ratio_3 = {0:int(round(Tn1*.25)),1:int(round(Tn1*.40)),2:int(round(Tn1*.35))} #new ratio dict
ratio_3
print(ratio_3)
sum(ratio_3.values())

{0: 2694, 1: 4310, 2: 3771}


10775

### ratio 4: 20 45 35


In [62]:
Tn2 = int(round(class0_rows/.45))
Tn2

9578

In [63]:
ratio_4 = {0:int(round(Tn2*.20)),1:int(round(Tn2*.45)),2:int(round(Tn2*.35))} #new ratio dict
ratio_4
print(ratio_4)
sum(ratio_4.values())

{0: 1916, 1: 4310, 2: 3352}


9578

In [45]:
X = data_wo3.tweet_ts
X.shape#[0]*.4

(7148,)

In [46]:
y = data_wo3.sentiment
y.shape

(7148,)

In [47]:
# Count vec
cv1 = CountVectorizer(ngram_range=(1, 3))
cv1

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [48]:
X_cv1 = genf.vectorizer_fit_transform(cv1,X)

In [49]:
X_cv1.shape

(7148, 102771)

#### SMOTE with ratio1

In [50]:
sm1 = SMOTE(random_state=42,sampling_strategy=ratio_1)
X_cv_res1, y_res1 = sm1.fit_resample(X_cv1, y)

In [51]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_cv_res1, y_res1, train_size=0.8, random_state=10)

In [52]:
genf.fit_models(X_train1, X_test1, y_train1, y_test1)

CLF report for RandomForestClassifier
              precision    recall  f1-score   support

           0       0.73      0.87      0.80       613
           1       0.69      0.73      0.71       889
           2       0.63      0.47      0.54       653

    accuracy                           0.69      2155
   macro avg       0.69      0.69      0.68      2155
weighted avg       0.69      0.69      0.68      2155

---

CLF report for GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.69      0.81      0.74       613
           1       0.66      0.79      0.71       889
           2       0.62      0.35      0.45       653

    accuracy                           0.66      2155
   macro avg       0.65      0.65      0.63      2155
weighted avg       0.65      0.66      0.64      2155

---



/Users/Aniket/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


CLF report for LogisticRegression
              precision    recall  f1-score   support

           0       0.76      0.86      0.81       613
           1       0.71      0.74      0.73       889
           2       0.62      0.52      0.57       653

    accuracy                           0.71      2155
   macro avg       0.70      0.71      0.70      2155
weighted avg       0.70      0.71      0.70      2155

---

CLF report for XGBClassifier
              precision    recall  f1-score   support

           0       0.98      0.88      0.93       613
           1       0.71      0.89      0.79       889
           2       0.79      0.59      0.67       653

    accuracy                           0.80      2155
   macro avg       0.83      0.79      0.80      2155
weighted avg       0.81      0.80      0.79      2155

---



#### SMOTE with ratio 2

In [53]:
sm2 = SMOTE(random_state=42,sampling_strategy=ratio_2)
X_cv_res2, y_res2 = sm2.fit_resample(X_cv1, y)

In [54]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_cv_res2, y_res2, train_size=0.8, random_state=10)

In [55]:
genf.fit_models(X_train2, X_test2, y_train2, y_test2)

CLF report for RandomForestClassifier
              precision    recall  f1-score   support

           0       0.68      0.60      0.64       428
           1       0.73      0.65      0.69       894
           2       0.62      0.73      0.67       833

    accuracy                           0.67      2155
   macro avg       0.67      0.66      0.66      2155
weighted avg       0.68      0.67      0.67      2155

---

CLF report for GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.59      0.58      0.58       428
           1       0.65      0.70      0.67       894
           2       0.56      0.52      0.54       833

    accuracy                           0.60      2155
   macro avg       0.60      0.60      0.60      2155
weighted avg       0.60      0.60      0.60      2155

---



/Users/Aniket/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


CLF report for LogisticRegression
              precision    recall  f1-score   support

           0       0.69      0.57      0.62       428
           1       0.72      0.71      0.72       894
           2       0.63      0.70      0.66       833

    accuracy                           0.68      2155
   macro avg       0.68      0.66      0.67      2155
weighted avg       0.68      0.68      0.68      2155

---

CLF report for XGBClassifier
              precision    recall  f1-score   support

           0       0.95      0.83      0.89       428
           1       0.71      0.87      0.78       894
           2       0.83      0.69      0.75       833

    accuracy                           0.79      2155
   macro avg       0.83      0.80      0.81      2155
weighted avg       0.80      0.79      0.79      2155

---



#### SMOTE with ratio3

In [56]:
sm3 = SMOTE(random_state=42,sampling_strategy=ratio_3)
X_cv_res3, y_res3 = sm3.fit_resample(X_cv1, y)

In [57]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_cv_res3, y_res3, train_size=0.8, random_state=10)

In [58]:
genf.fit_models(X_train3, X_test3, y_train3, y_test3)

CLF report for RandomForestClassifier
              precision    recall  f1-score   support

           0       0.66      0.82      0.73       528
           1       0.71      0.69      0.70       894
           2       0.61      0.52      0.56       733

    accuracy                           0.67      2155
   macro avg       0.66      0.68      0.67      2155
weighted avg       0.67      0.67      0.66      2155

---

CLF report for GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.65      0.73      0.69       528
           1       0.66      0.74      0.70       894
           2       0.58      0.45      0.51       733

    accuracy                           0.64      2155
   macro avg       0.63      0.64      0.63      2155
weighted avg       0.63      0.64      0.63      2155

---



/Users/Aniket/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


CLF report for LogisticRegression
              precision    recall  f1-score   support

           0       0.69      0.82      0.75       528
           1       0.73      0.73      0.73       894
           2       0.63      0.54      0.58       733

    accuracy                           0.69      2155
   macro avg       0.68      0.70      0.69      2155
weighted avg       0.68      0.69      0.68      2155

---

CLF report for XGBClassifier
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       528
           1       0.71      0.88      0.79       894
           2       0.80      0.64      0.71       733

    accuracy                           0.79      2155
   macro avg       0.83      0.79      0.80      2155
weighted avg       0.80      0.79      0.79      2155

---



In [ ]:
sm4 = SMOTE(random_state=42,sampling_strategy=ratio_4)
X_cv_res4, y_res4 = sm4.fit_resample(X_cv1, y)

X_train4, X_test4, y_train4, y_test4 = train_test_split(X_cv_res4, y_res4, train_size=0.8, random_state=10)

genf.fit_models(X_train4, X_test4, y_train4, y_test4)